In [328]:
%matplotlib inline
import numpy as np
from sklearn import svm
import pickle
from sklearn.metrics import accuracy_score
import torch
from sklearn.utils import shuffle
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torchvision.datasets as dsets
import torchvision.transforms as transforms

In [329]:
class Rnn(nn.Module):
    def __init__(self, input_dim, hidden_dim, layer_dim, output_dim):
        super(Rnn, self).__init__()
        self.hidden_dim=hidden_dim
        
        self.output_dim=output_dim
        self.layer_dim=layer_dim
        
        self.rnn=nn.RNN(input_dim,self.hidden_dim,bias=True,nonlinearity='relu')
        self.fc=nn.Linear(self.hidden_dim,self.output_dim)
    
    def forward(self,x):
        
        h0=torch.zeros(self.layer_dim,x.size(0),self.hidden_dim).requires_grad_()
        out,hn=self.rnn(x.unsqueeze(0),h0)
        output=self.fc(out[:,-1,:])
        return output

In [330]:
inn=open("test_data.pickle","rb")
test_data=pickle.load(inn)
inn=open("train_data.pickle","rb")
train_data=pickle.load(inn)
inn=open("test_labels.pickle","rb")
test_labels=pickle.load(inn)
inn=open("train_labels.pickle","rb")
train_labels=pickle.load(inn)

print("Train Data : ",np.shape(train_data))
print("Train Label : ",np.shape(train_labels))
print("Test Data : ",np.shape(test_data))
print("Test Label : ",np.shape(test_labels))


Train Data :  (199, 1001, 72)
Train Label :  (199,)
Test Data :  (50, 1001, 72)
Test Label :  (50,)


In [331]:
#convert time domain to frequecy domain

for i in range(np.size(train_data,axis=0)):
    train_data[i,:,:]=np.fft.ifft(train_data[i,:,:])

for i in range(np.size(test_data,axis=0)):
    test_data[i,:,:]=np.fft.ifft(test_data[i,:,:])


In [332]:
# Take the absolute of windowed signal

test_data=np.abs(test_data[:,5:40,:])
train_data=np.abs(train_data[:,5:40,:])
train_data=train_data/(np.max(train_data))
test_data=test_data/(np.max(test_data))



#train_data = np.reshape(train_data, [np.size(train_data, axis=0),np.size(train_data,axis=1) * np.size(train_data,axis=2)])
#test_data = np.reshape(test_data, [np.size(test_data, axis=0), np.size(test_data,axis=1)*np.size(test_data,axis=2)])


print("Train Data : ",np.shape(train_data))
print("Train Label : ",np.shape(train_labels))
print("Test Data : ",np.shape(test_data))
print("Test Label : ",np.shape(test_labels))


Train Data :  (199, 35, 72)
Train Label :  (199,)
Test Data :  (50, 35, 72)
Test Label :  (50,)


In [333]:
batch_size=10
iterations=30
epoch=int(iterations/(len(train_data)/batch_size))
print(epoch)
train_data, train_labels =shuffle(train_data, train_labels)

train_data=torch.tensor(train_data).float()
train_labels=torch.tensor(train_labels).float()
test_data=torch.tensor(test_data).float()
test_labels=torch.tensor(test_labels).float()

1


In [334]:
input_dim=72
hidden_dim=100
layer_dim=1
output_dim=2
model=Rnn(input_dim,hidden_dim,layer_dim,output_dim)

In [335]:
criterion=nn.CrossEntropyLoss()
Lr=0.001
optimizer = torch.optim.SGD(model.parameters(), lr=Lr)  


In [336]:

iter=0
for i in range (train_data.size(0)):

    
    output=model(train_data[i,:,:])
    lab=train_labels[i].unsqueeze(0)
    loss=criterion(output,lab.long())
    optimizer.zero_grad()
    iter+=1
    loss.backward()
    optimizer.step()
    for j in range(test_data.size(0)):
        outt=model(test_data[j,:,:])
        _,pred=torch.max(outt.data,1)
        correct = 0
        total = 0
        total += test_labels.size(0)
                
                # Total correct predictions
        correct += (pred == test_labels).sum()
            
        accuracy = 100 * correct.item() / total
    print('Iter: {}. Loss: {}. Accuracy: {}'.format(iter, loss.item(), accuracy)) 

RuntimeError: input.size(-1) must be equal to input_size. Expected 2520, got 72

In [ ]:
a=torch.tensor([[1,2],[2,3.2],[3.3,2.2]])
print(a)
a,b=torch.max(a,1)
print("\n\n",a)